In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
SWOT Pixel Cloud Plotter
========================

This script reads a reduced SWOT L2 pixel cloud NetCDF file, 
and creates two plots:

1. All pixel cloud classes labeled with their names.
2. Open water points (class 4) colored by height (WSE).

Basemap is provided via CartoDB Positron.

Author: Shayan Shirafkan
Date: 2025-11-05
License: MIT
"""

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import contextily as ctx

# -------------------------------
# Load reduced dataset
# -------------------------------
ds = xr.open_dataset('SWOT_15Khordad_small.nc')
lon = ds['longitude'].values
lat = ds['latitude'].values
h   = ds['height'].values
c   = ds['classification'].values

# -------------------------------
# Downsample for faster plotting
# -------------------------------
step = 10
lon = lon[::step]
lat = lat[::step]
h   = h[::step]
c   = c[::step]

# -------------------------------
# Class names mapping
# -------------------------------
class_dict = {
    1: "land",
    2: "land_near_water",
    3: "water_near_land",
    4: "open_water",
    5: "dark_water",
    6: "low_coh_water_near_land",
    7: "open_low_coh_water"
}

# -------------------------------
# Convert to GeoDataFrame
# -------------------------------
geometry = [Point(xy) for xy in zip(lon, lat)]
gdf = gpd.GeoDataFrame({'classification': c, 'height': h}, geometry=geometry, crs="EPSG:4326")
gdf = gdf.to_crs(epsg=3857)

# -------------------------------
# Plot 1: All classes labeled
# -------------------------------
plt.figure(figsize=(12, 8))
ax = plt.gca()
for class_id, name in class_dict.items():
    gdf_class = gdf[gdf['classification'] == class_id]
    gdf_class.plot(ax=ax, markersize=10, label=name, alpha=0.7)

ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.set_title("SWOT Pixel Cloud - All Classes (Labels) 15 Khordad Dam")
ax.set_axis_off()
plt.legend()
plt.show()

# -------------------------------
# Plot 2: Open water colored by height
# -------------------------------
plt.figure(figsize=(12, 8))
ax = plt.gca()
gdf_water = gdf[gdf['classification'] == 4]
gdf_water.plot(ax=ax, column='height', cmap='Blues', markersize=10, legend=True, alpha=0.8)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.set_title("SWOT Pixel Cloud - Open Water Height 15 Khordad Dam (Iran)")
ax.set_axis_off()
plt.show()
